<a href="https://colab.research.google.com/github/Paulina-Lopez/ProjectN-3-TET/blob/main/ProjectN%C2%B03-TET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pwd

/content


In [2]:
!pip install spark
!pip install pyspark
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 554.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58749 sha256=c8fe43089af5db3c6f7dff803df1fed32de2227d8051aff3547c274449bf7b06
  Stored in directory: /root/.cache/pip/wheels/63/88/77/b4131110ea4094540f7b47c6d62a649807d7e94800da5eab0b
Successfully built spark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=393a6abfbcbda66698d012959c594c3fc42aef6bbc9b53b2bb5b3c50b20f5d2b
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark=SparkSession.builder.appName('data_processing').getOrCreate()

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
df = spark.read.csv('/content/gdrive/MyDrive/mplopezs-tet/bigdata/Casos_positivos_de_COVID-19_en_Colombia-100K.csv', inferSchema=True, header=True)

In [10]:
df.columns

['fecha reporte web',
 'ID de caso',
 'Fecha de notificación',
 'Código DIVIPOLA departamento',
 'Nombre departamento',
 'Código DIVIPOLA municipio',
 'Nombre municipio',
 'Edad',
 'Unidad de medida de edad',
 'Sexo',
 'Tipo de contagio',
 'Ubicación del caso',
 'Estado',
 'Código ISO del país',
 'Nombre del país',
 'Recuperado',
 'Fecha de inicio de síntomas',
 'Fecha de muerte',
 'Fecha de diagnóstico',
 'Fecha de recuperación',
 'Tipo de recuperación',
 'Pertenencia étnica',
 'Nombre del grupo étnico']

In [11]:
df.printSchema()

root
 |-- fecha reporte web: string (nullable = true)
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: string (nullable = true)
 |-- Código DIVIPOLA departamento: integer (nullable = true)
 |-- Nombre departamento: string (nullable = true)
 |-- Código DIVIPOLA municipio: integer (nullable = true)
 |-- Nombre municipio: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Unidad de medida de edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo de contagio: string (nullable = true)
 |-- Ubicación del caso: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Código ISO del país: integer (nullable = true)
 |-- Nombre del país: string (nullable = true)
 |-- Recuperado: string (nullable = true)
 |-- Fecha de inicio de síntomas: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha de diagnóstico: string (nullable = true)
 |-- Fecha de recuperación: string (nullable = true)
 |-- Tipo de r

In [14]:
df.show(5)

+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+------+-------------------+---------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+------------------+-----------------------+
|fecha reporte web|ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Ubicación del caso|Estado|Código ISO del país|Nombre del país|Recuperado|Fecha de inicio de síntomas|Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|
+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+---------

In [15]:
df.select('Edad', 'Nombre municipio').show(5)

+----+----------------+
|Edad|Nombre municipio|
+----+----------------+
|  19|          BOGOTA|
|  34|            BUGA|
|  50|        MEDELLIN|
|  55|        MEDELLIN|
|  25|        MEDELLIN|
+----+----------------+
only showing top 5 rows



In [17]:
df = df.withColumnRenamed("ID de caso", "ID")
df = df.withColumnRenamed("fecha reporte web", "Fecha de reporte web")
df.columns

['Fecha de reporte web',
 'ID',
 'Fecha de notificación',
 'Código DIVIPOLA departamento',
 'Nombre departamento',
 'Código DIVIPOLA municipio',
 'Nombre municipio',
 'Edad',
 'Unidad de medida de edad',
 'Sexo',
 'Tipo de contagio',
 'Ubicación del caso',
 'Estado',
 'Código ISO del país',
 'Nombre del país',
 'Recuperado',
 'Fecha de inicio de síntomas',
 'Fecha de muerte',
 'Fecha de diagnóstico',
 'Fecha de recuperación',
 'Tipo de recuperación',
 'Pertenencia étnica',
 'Nombre del grupo étnico']

In [22]:
df = df.withColumn("Tiempo hasta diagnóstico", (df["Fecha de diagnóstico"] - df["Fecha de inicio de síntomas"]))
df.columns

['Fecha de reporte web',
 'ID',
 'Fecha de notificación',
 'Código DIVIPOLA departamento',
 'Nombre departamento',
 'Código DIVIPOLA municipio',
 'Nombre municipio',
 'Edad',
 'Unidad de medida de edad',
 'Sexo',
 'Tipo de contagio',
 'Ubicación del caso',
 'Estado',
 'Código ISO del país',
 'Nombre del país',
 'Recuperado',
 'Fecha de inicio de síntomas',
 'Fecha de muerte',
 'Fecha de diagnóstico',
 'Fecha de recuperación',
 'Tipo de recuperación',
 'Pertenencia étnica',
 'Nombre del grupo étnico',
 'Tiempo hasta diagnóstico']

In [23]:
df = df.drop("Tiempo hasta diagnóstico")

In [24]:
df.filter((df['Sexo']=='M')&(df['Edad']>50)).show()

+--------------------+---+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+------+-------------------+--------------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+------------------+-----------------------+
|Fecha de reporte web| ID|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Ubicación del caso|Estado|Código ISO del país|     Nombre del país|Recuperado|Fecha de inicio de síntomas|Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|
+--------------------+---+---------------------+----------------------------+-------------------+-------------------------+----------------+----+-----------

In [26]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [28]:
millenium_udf = udf(lambda edad: "2do Milenio" if edad <= 24 else "1er Milenio", StringType())

In [29]:
df.withColumn("Milenio nacimiento", millenium_udf(df.Edad)).select('Edad', 'Milenio nacimiento').show(10, False)

+----+------------------+
|Edad|Milenio nacimiento|
+----+------------------+
|19  |2do Milenio       |
|34  |1er Milenio       |
|50  |1er Milenio       |
|55  |1er Milenio       |
|25  |1er Milenio       |
|27  |1er Milenio       |
|85  |1er Milenio       |
|22  |2do Milenio       |
|28  |1er Milenio       |
|36  |1er Milenio       |
+----+------------------+
only showing top 10 rows



In [31]:
from pyspark.sql.functions import col, count

top_10_casos_por_departamento = df.groupBy("Nombre departamento").agg(count('*').alias('Casos')).orderBy(col('Casos').desc())
top_10_casos_por_departamento.show(10)

+-------------------+-----+
|Nombre departamento|Casos|
+-------------------+-----+
|             BOGOTA|30016|
|       BARRANQUILLA|13065|
|          ATLANTICO|10994|
|              VALLE|10404|
|          CARTAGENA| 8333|
|          ANTIOQUIA| 4554|
|             NARIÑO| 3520|
|       CUNDINAMARCA| 2827|
|           AMAZONAS| 2317|
|              CHOCO| 1636|
+-------------------+-----+
only showing top 10 rows



In [32]:
top_10_casos_por_ciudad = df.groupBy("Nombre municipio").agg(count('*').alias('Casos')).orderBy(col('Casos').desc())
top_10_casos_por_ciudad.show(10)

+----------------+-----+
|Nombre municipio|Casos|
+----------------+-----+
|          BOGOTA|30016|
|    BARRANQUILLA|13065|
|       CARTAGENA| 8333|
|            CALI| 7747|
|         SOLEDAD| 6233|
|         LETICIA| 2194|
|        MEDELLIN| 2137|
|          TUMACO| 1501|
|    BUENAVENTURA| 1453|
|          QUIBDO| 1367|
+----------------+-----+
only showing top 10 rows



In [33]:
top_10_casos_por_fecha = df.groupBy("Fecha de diagnóstico").agg(count('*').alias('Casos')).orderBy(col('Casos').desc())
top_10_casos_por_fecha.show(10)

+--------------------+-----+
|Fecha de diagnóstico|Casos|
+--------------------+-----+
|   26/6/2020 0:00:00| 4390|
|   27/6/2020 0:00:00| 4019|
|   28/6/2020 0:00:00| 3580|
|   25/6/2020 0:00:00| 3381|
|   19/6/2020 0:00:00| 3053|
|   18/6/2020 0:00:00| 3040|
|   23/6/2020 0:00:00| 3031|
|   22/6/2020 0:00:00| 2938|
|   21/6/2020 0:00:00| 2781|
|   24/6/2020 0:00:00| 2564|
+--------------------+-----+
only showing top 10 rows



In [35]:
distribucion_casos_por_edad = df.groupBy("Edad").agg(count('*').alias('Casos')).orderBy(col('Edad').desc())
distribucion_casos_por_edad.show(distribucion_casos_por_edad.count())

+----+-----+
|Edad|Casos|
+----+-----+
| 104|    1|
| 103|    2|
| 102|    2|
| 101|    1|
| 100|   10|
|  99|   11|
|  98|   12|
|  97|   14|
|  96|   23|
|  95|   32|
|  94|   53|
|  93|   51|
|  92|   70|
|  91|   91|
|  90|  109|
|  89|  117|
|  88|  146|
|  87|  184|
|  86|  182|
|  85|  224|
|  84|  213|
|  83|  262|
|  82|  287|
|  81|  275|
|  80|  346|
|  79|  368|
|  78|  346|
|  77|  348|
|  76|  361|
|  75|  397|
|  74|  436|
|  73|  479|
|  72|  533|
|  71|  501|
|  70|  593|
|  69|  583|
|  68|  614|
|  67|  688|
|  66|  701|
|  65|  736|
|  64|  800|
|  63|  867|
|  62|  932|
|  61|  965|
|  60| 1069|
|  59| 1085|
|  58| 1155|
|  57| 1252|
|  56| 1325|
|  55| 1291|
|  54| 1290|
|  53| 1307|
|  52| 1402|
|  51| 1327|
|  50| 1423|
|  49| 1467|
|  48| 1475|
|  47| 1401|
|  46| 1398|
|  45| 1492|
|  44| 1488|
|  43| 1654|
|  42| 1620|
|  41| 1783|
|  40| 2050|
|  39| 1985|
|  38| 2098|
|  37| 2132|
|  36| 2175|
|  35| 2292|
|  34| 2310|
|  33| 2371|
|  32| 2362|
|  31| 2569|

In [36]:
top_10_fallecidos_por_ciudad= df.filter(col("Estado") == "Fallecido").groupBy('Nombre municipio').agg(count('*').alias('Fallecidos')).orderBy(col('Fallecidos').desc())
top_10_fallecidos_por_ciudad.show(10)

+----------------+----------+
|Nombre municipio|Fallecidos|
+----------------+----------+
|          BOGOTA|       962|
|    BARRANQUILLA|       926|
|         SOLEDAD|       406|
|       CARTAGENA|       392|
|            CALI|       365|
|    BUENAVENTURA|       126|
|       SINCELEJO|        90|
|         LETICIA|        90|
|         MALAMBO|        89|
|     SANTA MARTA|        69|
+----------------+----------+
only showing top 10 rows



In [37]:
df.createOrReplaceTempView("covid_info_tbl")

In [44]:
from pyspark.sql.functions import col, count

top_10_casos_por_departamento = spark.sql("""
    SELECT `Nombre departamento`, COUNT(*) AS Casos
    FROM covid_info_tbl
    GROUP BY `Nombre departamento`
    ORDER BY Casos DESC
""")
top_10_casos_por_departamento.show(10)

+-------------------+-----+
|Nombre departamento|Casos|
+-------------------+-----+
|             BOGOTA|30016|
|       BARRANQUILLA|13065|
|          ATLANTICO|10994|
|              VALLE|10404|
|          CARTAGENA| 8333|
|          ANTIOQUIA| 4554|
|             NARIÑO| 3520|
|       CUNDINAMARCA| 2827|
|           AMAZONAS| 2317|
|              CHOCO| 1636|
+-------------------+-----+
only showing top 10 rows



In [45]:
top_10_casos_por_ciudad = spark.sql("""
    SELECT `Nombre municipio`, COUNT(*) AS Casos
    FROM covid_info_tbl
    GROUP BY `Nombre municipio`
    ORDER BY Casos DESC
""")
top_10_casos_por_ciudad.show(10)

+----------------+-----+
|Nombre municipio|Casos|
+----------------+-----+
|          BOGOTA|30016|
|    BARRANQUILLA|13065|
|       CARTAGENA| 8333|
|            CALI| 7747|
|         SOLEDAD| 6233|
|         LETICIA| 2194|
|        MEDELLIN| 2137|
|          TUMACO| 1501|
|    BUENAVENTURA| 1453|
|          QUIBDO| 1367|
+----------------+-----+
only showing top 10 rows



In [46]:
top_10_casos_por_fecha = spark.sql("""
    SELECT `Fecha de diagnóstico`, COUNT(*) AS Casos
    FROM covid_info_tbl
    GROUP BY `Fecha de diagnóstico`
    ORDER BY Casos DESC
""")
top_10_casos_por_fecha.show(10)

+--------------------+-----+
|Fecha de diagnóstico|Casos|
+--------------------+-----+
|   26/6/2020 0:00:00| 4390|
|   27/6/2020 0:00:00| 4019|
|   28/6/2020 0:00:00| 3580|
|   25/6/2020 0:00:00| 3381|
|   19/6/2020 0:00:00| 3053|
|   18/6/2020 0:00:00| 3040|
|   23/6/2020 0:00:00| 3031|
|   22/6/2020 0:00:00| 2938|
|   21/6/2020 0:00:00| 2781|
|   24/6/2020 0:00:00| 2564|
+--------------------+-----+
only showing top 10 rows



In [47]:
top_10_casos_por_edad = spark.sql("""
    SELECT `Edad`, COUNT(*) AS Casos
    FROM covid_info_tbl
    GROUP BY `Edad`
    ORDER BY Casos DESC
""")
top_10_casos_por_edad.show(10)

+----+-----+
|Edad|Casos|
+----+-----+
|  30| 2735|
|  29| 2611|
|  31| 2569|
|  28| 2540|
|  27| 2494|
|  26| 2436|
|  33| 2371|
|  32| 2362|
|  25| 2335|
|  34| 2310|
+----+-----+
only showing top 10 rows



In [48]:
top_10_fallecidos_por_ciudad = spark.sql("""
    SELECT `Nombre municipio`, COUNT(*) AS Fallecidos
    FROM covid_info_tbl
    WHERE Estado = 'Fallecido'
    GROUP BY `Nombre municipio`
    ORDER BY Fallecidos DESC
""")
top_10_fallecidos_por_ciudad.show(10)

+----------------+----------+
|Nombre municipio|Fallecidos|
+----------------+----------+
|          BOGOTA|       962|
|    BARRANQUILLA|       926|
|         SOLEDAD|       406|
|       CARTAGENA|       392|
|            CALI|       365|
|    BUENAVENTURA|       126|
|       SINCELEJO|        90|
|         LETICIA|        90|
|         MALAMBO|        89|
|     SANTA MARTA|        69|
+----------------+----------+
only showing top 10 rows



In [52]:
top_10_casos_por_departamento.toPandas().to_csv('/content/gdrive/MyDrive/mplopezs-tet/bigdata/top_10_casos_por_departamento.csv')

In [53]:
top_10_casos_por_ciudad.toPandas().to_csv('/content/gdrive/MyDrive/mplopezs-tet/bigdata/top_10_casos_por_ciudad.csv')

In [54]:
top_10_casos_por_fecha.toPandas().to_csv('/content/gdrive/MyDrive/mplopezs-tet/bigdata/top_10_casos_por_fecha.csv')

In [55]:
top_10_casos_por_edad.toPandas().to_csv('/content/gdrive/MyDrive/mplopezs-tet/bigdata/top_10_casos_por_edad.csv')

In [56]:
top_10_fallecidos_por_ciudad.toPandas().to_csv('/content/gdrive/MyDrive/mplopezs-tet/bigdata/top_10_fallecidos_por_ciudad.csv')

In [ ]:
top_10_casos_por_departamento.write.format('csv').mode('overwrite').save('s3://mplopezs-tet/bigdata/top_10_casos_por_departamento.csv')

In [ ]:
top_10_casos_por_ciudad.write.format('csv').mode('overwrite').save('s3://mplopezs-tet/bigdata/top_10_casos_por_ciudad.csv')

In [ ]:
top_10_casos_por_fecha.write.format('csv').mode('overwrite').save('s3://mplopezs-tet/bigdata/top_10_casos_por_fecha.csv')

In [ ]:
top_10_casos_por_edad.write.format('csv').mode('overwrite').save('s3://mplopezs-tet/bigdata/top_10_casos_por_edad.csv')